In [10]:
import os
import dotenv
from wine_rater import logger

In [6]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [2]:
os.chdir('..')

In [23]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metrics_path: Path
    target_column: str
    mlflow_uri: str

In [24]:
from wine_rater.constants import *
from wine_rater.entity.config_entity import DataIngestionConfig, DataTransformationConfig, DataValidationConfig, ModelTrainerConfig
from wine_rater.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metrics_path=config.metrics_path,
            target_column=schema.name,
            mlflow_uri=os.environ['MLFLOW_TRACKING_URI']
        )

        return model_evaluation_config


In [29]:
from urllib.parse import urlparse
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd
import mlflow

from wine_rater.utils.common import save_json

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluate(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)

        return { "rmse": rmse, "mae": mae, "r2": r2 }
    
    def log_to_mlflow(self):
        model = joblib.load(self.config.model_path)
        test_data = pd.read_csv(self.config.test_data_path)

        test_x = test_data.drop(self.config.target_column, axis=1)
        test_y = test_data[self.config.target_column]

        mlflow.set_registry_uri = self.config.mlflow_uri
        tracking_url_type_store = urlparse(mlflow.get_registry_uri()).scheme

        with mlflow.start_run():
            predicted = model.predict(test_x)

            scores = self.evaluate(test_y, predicted)
            save_json(path=Path(self.config.metrics_path), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metrics(scores)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNet")
            else:
                mlflow.sklearn.log_model(model, "model")


In [31]:
try:
    config_manager = ConfigurationManager()
    model_evaluation_config = config_manager.get_model_evaluation_config()

    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.log_to_mlflow()
except Exception as e:
    logger.error(e)
    raise e

[2023-12-19 17:32:16,042: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-19 17:32:16,044: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-19 17:32:16,046: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-19 17:32:16,048: INFO: common: created directory at: artifacts]
[2023-12-19 17:32:16,051: INFO: common: created directory at: artifacts/model_evaluation]
[2023-12-19 17:32:16,059: ERROR: 2111813090: Run with UUID 8648fe3ec85a4279b0c52075609ee775 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True]


Exception: Run with UUID 8648fe3ec85a4279b0c52075609ee775 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True